# County Level COVID-19 Reports
This notebook relies on data gathered by Johns Hopkins University to present charts illustrating the progression of COVID-19 at the County level for the state specified.
The code is not the most efficient nor the most elegant.  Yes, I could have streamlined it quite a bit, but I have a day job, too.  It provides some nice examples of
annotations in matplotlib.  It doesn't really need any fancy packages from Python, just some common ones: Pandas, matplotlib.  Seaborn is referenced, but I think I abandoned using it in favor of matplotlib.

David West, IBM 2 June 2020

### Specify the name of the state in the next box in the form s = "Nameofstate"
### Then run all cells.


In [ ]:
#s = "Massachusetts"

s = "Virginia"
s = "Pennsylvania"

s = "Georgia"
s = "Illinois"
s = "New York"
#s = "Connecticut"
s = "Illinois"
s = "California"
s = "Florida"

 



In [ ]:
# This will print at the bottom of each chart.  Feel free to put your name here.
myname = "David A. West, IBM"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from datetime import date
from datetime import timedelta
now = (str(datetime.now()))
nowdate = now[0:10] 
print(nowdate)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 65em; }</style>"))


import matplotlib.ticker
class MyLocator(matplotlib.ticker.AutoMinorLocator):
    def __init__(self, n=14):
        super().__init__(n=n)
matplotlib.ticker.AutoMinorLocator = MyLocator        



# Read in the time series files published by Johns Hopkins
### There are many quality issues with this data.  The column names were changed in the 3/23 dataset.  Then on 3/25, they retroactively made changes to the 3/22 dataset.  Values in Province/State are inconsistent in the US data.  At times they have a City, ST.  Other times the name of the State is spelled out completely.  This needs to be cleaned up.  DAW 3-25-2020

### Found a coding error that caused the 3/22 data to be read and appended 3 times but no 3/23 or 3/24 data was read.  Error fixed.  DAW 3-25-2020
### 3/22 Data is first in this set of tables that contains the Admin2 field which has the County in it for the US.

In [ ]:
tshx1 = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
tshx2 = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
tshx3 = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
# READ CONFIRMED COUNTS
c = pd.read_csv(tshx1, error_bad_lines=False)
c = c[c["Province_State"] == s]
c = c.drop(['UID',"iso2","iso3","code3","FIPS","Lat","Long_","Combined_Key","Country_Region" ], axis = 1, inplace=False)
c = c[(c["Admin2"]!="Bronx") & (c["Admin2"]!="Queens")& (c["Admin2"]!="Kings")& (c["Admin2"]!="Richmond")& (c["Admin2"]!="Unassigned") & (c["Admin2"]!="Out of NY")]
# READ DEATH COUNTS
d = pd.read_csv(tshx2, error_bad_lines=False)
d = d[d["Province_State"] == s]
d = d.drop(['UID',"iso2","iso3","code3","FIPS","Lat","Long_","Combined_Key","Country_Region" ,"Population"], axis = 1, inplace=False)
d = d[(d["Admin2"]!="Bronx") & (d["Admin2"]!="Queens")& (d["Admin2"]!="Kings")& (d["Admin2"]!="Richmond")& (d["Admin2"]!="Unassigned") & (d["Admin2"]!="Out of NY")]
 

In [ ]:
c2=(c.set_index(["Admin2",'Province_State'])
   .stack()
   .rename_axis(['Admin2','Province/State', 'Date'])
   .reset_index(name='Confirmed')
)

c2.set_index(['Admin2','Province/State','Date'])
c2["Date"] = pd.to_datetime(c2['Date'])
c2["ConfDelta"] = c2["Confirmed"].diff()
#c2 = c2.loc[(c2['ConfDelta'] < 0),'test']=0
# Set first period equal to 0 - would be negative or NaN based on the structure of the data.
c2.loc[c2['ConfDelta'] < 0, 'ConfDelta'] = 0
c2 = c2.fillna(0)


In [ ]:
c2.head()

In [ ]:
d2=(d.set_index(["Admin2",'Province_State'])
   .stack()
   .rename_axis(['Admin2','Province/State', 'Date'])
   .reset_index(name='Deaths')
)

d2.set_index(['Admin2','Province/State','Date'])
d2["Date"] = pd.to_datetime(d2['Date'])
d2["DeathDelta"] = d2["Deaths"].diff()

# Set first period equal to 0 - would be negative or NaN based on the structure of the data.
d2.loc[d2['DeathDelta'] < 0, 'DeathDelta'] = 0
d2 = d2.fillna(0)


In [ ]:
d2.head()

In [ ]:
#State data
sd = c2.merge(d2, on=['Province/State','Admin2','Date'])
sd['Admin2'] =sd['Admin2'].str.replace('.','')
sd = sd[sd['Admin2']!="Out of FL"]

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 65em; }</style>"))

# Add Population Data

In [ ]:
#Population at the county level for all US counties obtained from the US Census Bureau website.  Data is the 2019 population estimate
# When the data are obtained, all counties are initially read, then the last line subsets the data for only the desired state.
# The data are cleaned up and new fields for merging with other data are created.

# https://github.com/DavidAWest/COVID_Coding/raw/master/USCensus_County_Pop_2019.csv
popin = "https://github.com/DavidAWest/COVID_Coding/raw/master/USCensus_County_Pop_2019.csv"
cpop = pd.read_csv(popin).fillna(0).replace({"County_state" : {"County" : "", "." : ""}})#.rename(columns={"County_state": "Admin2"})
cpop['Admin2'] = cpop['County_state'].str.replace(' County', '')
cpop['Admin2'] =cpop['Admin2'].str.replace('.','')
cpop['Admin2'] = cpop["Admin2"].str.split(',').str[0]
cpop['Province/State'] = cpop["County_state"].str.split(',').str[1].str.lstrip()
cpop['Province/State'].unique()
cpop = cpop[cpop['Province/State'] == s]

In [ ]:
cpop

In [ ]:
# County Land Area from US CENSUS BUREAU https://www2.census.gov/library/publications/2011/compendia/usa-counties/excel/LND01.xls
# https://www2.census.gov/library/publications/2011/compendia/usa-counties/excel/LND01.xls
# https://www.census.gov/library/publications/2011/compendia/usa-counties-2011.html#LND
# https://www.census.gov/library/publications/2011/compendia/usa-counties-2011/file-layout.html
# LND110200D = Land Area in sq mi
# LND210200D = Water Area in sq mi
# LND010200D = Total Area in sq mi
# STCOU = Code column header - FIPS State and County Code stored in this column (Excel spreadhseet column B) from rows 2 to 3197.
# Areaname = UNITED STATES | ALABAMA | Autauga, AL
from pandas import DataFrame

lndin = "https://github.com/DavidAWest/COVID_Coding/raw/master/Census%202012%20County%20Data%20Land%20Area.csv"
lnd = pd.read_csv(lndin,usecols=['Areaname','STCOU','LND110200D']).fillna(0)# .replace({"County_state" : {"County" : "", "." : ""}})#.rename(columns={"County_state": "Admin2"})
#x = lnd.Areaname.unique().tolist()
#xdf = DataFrame(x ,columns=["Areaname"]) #.toframe().to_csv('CensusAreaName.csv', index=False)
#xdf.to_csv('CensusAreaName.csv',index=False)

stnamesfile = "https://github.com/DavidAWest/COVID_Coding/raw/master/CensusAreaName.csv"
stname = pd.read_csv(stnamesfile, error_bad_lines=False, usecols = ['Areaname','Admin2','StateAbrev','State'])
sd.head()

In [ ]:
lnd

In [ ]:
cpop.head()

In [ ]:
# combine the county level data for the state with the population data.
sd = sd.merge(cpop, on="Admin2")

In [ ]:
sd

# Create new columns for analysis

In [ ]:
#Confirmed cases per 10,000 population
sd['Conf_per_10K'] = 10000* sd['Confirmed'] / sd['POP_2019']
# Daily increase per 10,000 pop
sd['Conf_inc_per_10K'] = 10000 * sd['ConfDelta']/sd['POP_2019']
# Deaths per 10K
sd['Deaths_per_10K'] = 10000 * sd['Deaths']/sd['POP_2019']
countylist = sd[sd['Date']==sd['Date'].max()].sort_values(by=['Conf_per_10K'])
countylist.head()

In [ ]:
# list of counties in order of their Confirmed Cases Per 10K - To be used later in looping through each county.
counties = countylist['Admin2'].unique()


# Per Capita (10K) Plots
## This plots the per capita infection rate daily for each county.  Some states have WAY TOO MANY counties for this to be very useful.
## Texas and Georgia have lots of counties.

In [ ]:
import itertools
# y is the starting point for annotations in the chart and represents the vertical distance from the bottom of the chart as a percentage of the chart.
y= 0.20
# x is the annotation horizontal position where the annotations will begin.
x= 0.15

# This is the sequence that the colors will loop through.  Edit it as you see fit.  It will repeat.
colors = itertools.cycle(["maroon","dodgerblue","darkblue","olive"  ,"red","aqua","goldenrod","blue","tomato","magenta","palegreen","deepskyblue","deeppink","darkorchid","turquoise", "indigo",
                          "violet","cadetblue","chartreuse","saddlebrown"])

fig=plt.figure(figsize=(20, 8), dpi= 80, facecolor='w', edgecolor='k')#.legend( prop={'size': 18})
# loop through each county name subsetting the data for the county and plot that county's line and annotate the chart with the name of the county and it's infection rate.
for county in counties:
    cd = sd[sd['Admin2']==county].copy().fillna(0)


    
    c=next(colors)
    plt.plot( 'Date', 'Conf_per_10K',data=cd, marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
    plt.annotate( county + ": "+ cd["Conf_per_10K"].iloc[(-1)].round(3).astype(str),  color=c,size=14, 
                 xy=(x, y), 
                 xytext=(x, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    y+=0.023
    
    if y > 0.9:
        y = 0.20
        x += 0.12

title = "COVID-19 Infections Per Capita (per 10,000) State of " + s 

plt.title(title, fontsize=24)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Count of Cases COVID-19 Per 10K', fontsize=24)
plt.rcParams['xtick.labelsize'] =16
plt.figtext(0.90, 0.01, 'Data gathered by Johns Hopkins University https://github.com/CSSEGISandData ' + now, horizontalalignment='right')
plt.figtext(0.10, 0.01, myname)
plt.tick_params(axis='both', which='minor', labelsize=16)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.grid(which='major', axis='both', linestyle='--',linewidth='2')
plt.grid(which='minor', axis='both', linestyle=':', linewidth='2', color='white')
plt.xticks(fontsize=14, rotation=0)
matplotlib.ticker.AutoMinorLocator(n=2)
plt.minorticks_on()
plt.grid( b=True,which='minor', axis='x', color='black', linestyle='-',alpha=0.1)
fn = s + '__ALL_counties_inf_rate_' + nowdate +'.jpg'
plt.savefig(fn)

plt.show()

In [ ]:
# the order of the counties list object determines the order that the charts are produced...since the desired order is different from the previous chart, I rebuild the list here.
countylist = sd[sd['Date']==sd['Date'].max()].sort_values(by=['Confirmed'])
counties = countylist['Admin2'].unique()


# PLOTS - COUNTY LEVEL - COUNTS and Moving Averages

In [ ]:
# PLOT 3,5 and 7 day moving averages

import itertools
y= 0.20
colors = itertools.cycle(["deepskyblue","darkblue","green","red"]) #,

for county in counties:
    cd = sd[sd['Admin2']==county].copy().fillna(0)
    cd["3 day moving"] = cd["ConfDelta"].rolling(window=3).mean()
    cd["5 day moving"] = cd["ConfDelta"].rolling(window=5).mean()
    cd["7 day moving"] = cd["ConfDelta"].rolling(window=7).mean()
    
    CONF = cd["Confirmed"].iloc[(-1)].astype(str)
    CONFPC = cd["Conf_per_10K"].iloc[(-1)].round(1).astype(str)
    DEAD = cd["Deaths"].iloc[(-1)].astype(str)
    CFR = (100*cd["Deaths"].iloc[(-1)] / cd["Confirmed"].iloc[(-1)]).round(2).astype(str)
    
    if cd["3 day moving"].iloc[(-2)] != 0:
        dt3 = (100*(cd["3 day moving"].iloc[(-1)]-cd["3 day moving"].iloc[(-2)])/cd["3 day moving"].iloc[(-2)]).round(3).astype(str)
    else:
        dt3 = "0"
    if cd["5 day moving"].iloc[(-2)] != 0:
        dt5 = (100*(cd["5 day moving"].iloc[(-1)]-cd["5 day moving"].iloc[(-2)])/cd["5 day moving"].iloc[(-2)]).round(3).astype(str)
    else:
        dt5 = "0"
    if cd["7 day moving"].iloc[(-2)] != 0:
        dt7 = (100*(cd["7 day moving"].iloc[(-1)]-cd["7 day moving"].iloc[(-2)])/cd["7 day moving"].iloc[(-2)]).round(3).astype(str)
    else:
        dt7 = "0"
    two_week_change = cd["7 day moving"].iloc[(-1)] - cd["7 day moving"].iloc[(-14)] 

    
    y= 0.30
    x1 = 0.15
    x2 = 0.4
    # The plots are getting too wide...and the dates are jumbled as a result.  This subsets the data starting with 2020-03-01.  If you want the full data, then change the date
    cd1 = cd[cd['Date']>='2020-03-01']
    
    # ANNOTATIONS
    fig=plt.figure(figsize=(20, 8), dpi= 80, facecolor='w', edgecolor='k')#.legend( prop={'size': 18})
    plt.annotate(CONF,  color="black",size=20, 
                 xy=(x2, 0.80), 
                 xytext=(x2, 0.80),horizontalalignment = 'right',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    plt.annotate(CONFPC,  color="black",size=20, 
                 xy=(x2, 0.77), 
                 xytext=(x2, 0.77),horizontalalignment = 'right',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    plt.annotate(DEAD,  color="black",size=20, 
                 xy=(x2, 0.74), 
                 xytext=(x2, 0.74),horizontalalignment = 'right',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    plt.annotate(CFR + "%",  color="black",size=20, 
                 xy=(x2, 0.71), 
                 xytext=(x2, 0.71),horizontalalignment = 'right',
                 xycoords = 'figure fraction',textcoords='figure fraction')

    
    plt.annotate("Confirmed: ",  color="black",size=20, 
                 xy=(x1, 0.80), 
                 xytext=(x1, 0.80),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    plt.annotate("Confirmed per 10K: ",  color="black",size=20, 
                 xy=(x1, 0.77), 
                 xytext=(x1, 0.77),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    plt.annotate("Deaths: ",  color="black",size=20, 
                 xy=(x1, 0.74), 
                 xytext=(x1, 0.74),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    plt.annotate("Case Fatality Rate: ",  color="black",size=20, 
                 xy=(x1, 0.71), 
                 xytext=(x1, 0.71),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')

    c=next(colors)
    plt.plot( 'Date', 'ConfDelta',data=cd1, marker='', markerfacecolor=c, markersize=12, color=c,linewidth=2)
    plt.annotate("Daily Increase: "+ cd1["ConfDelta"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    c=next(colors)
    y+=0.035
    plt.plot( 'Date', "3 day moving", data = cd1 , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=3)
    plt.annotate("3 day moving average: "+ cd1["3 day moving"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt3 +"% )",  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    c=next(colors)
    plt.plot( 'Date', "5 day moving", data = cd1 , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)
    y+=0.035
    plt.annotate("5 day moving average: "+ cd1["5 day moving"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt5 +"% )",  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    c=next(colors)
    plt.plot( 'Date', "7 day moving", data = cd1 , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=6)
    y+=0.035
    plt.annotate("7 day moving average: "+ cd1["7 day moving"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt7 +"% )",  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    y+=0.045
    plt.annotate("2 Week Change to 7-day moving: "+ two_week_change.round(3).astype(str) ,  color='black' ,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')

    title = "COVID-19 Daily Positive Infections: " + county + " County, " + s 

    plt.title(title, fontsize=24)
    plt.xlabel('Date', fontsize=20)
    plt.ylabel('Daily New Cases of COVID-19', fontsize=24)
    plt.rcParams['xtick.labelsize'] =22
    plt.figtext(0.90, 0.01, 'Data gathered by Johns Hopkins University https://github.com/CSSEGISandData ' + now, horizontalalignment='right')
    plt.figtext(0.10, 0.01, myname)
    plt.tick_params(axis='both', which='minor', labelsize=22)
    plt.tick_params(axis='both', which='major', labelsize=22)
    plt.grid(which='major', axis='both', linestyle='--',linewidth='2')
    plt.grid(which='minor', axis='both', linestyle=':', linewidth='2', color='white')
    plt.xticks(fontsize=14, rotation=0)
    
    matplotlib.ticker.AutoMinorLocator(n=2)
    plt.minorticks_on()
    plt.grid( b=True,which='minor', axis='x', color='black', linestyle='-',alpha=0.1)
    # This next section creates JPGs of each chart.  They will be stored wherever you are currently executing the notebook.  To change this, insert your path information as part of fn.
    fn = s + '-' + county + '_County_case_count_' + nowdate +'.jpg'
    plt.savefig(fn)

    plt.show()
    

In [ ]:

sd2 = sd.groupby(["Province/State_x","Date"])['Confirmed', 'ConfDelta', 'Deaths', 'DeathDelta'].sum().reset_index()
sd2.head()

In [ ]:
#an imperfect method for estimating recoveries based on the average recovery time.
# Lots of assumptions about the data are made - assumes that the date is the date that the patient got the disease. (We know that's not always true)
# Assumes that all patients recover or die by the period...28 days.  A very conservative recovery period.  Some are faster, some are rarely slower.
sd3 = sd.groupby(["Admin2","Date"])['Confirmed', 'ConfDelta', 'Deaths', 'DeathDelta'].sum().reset_index()
sd3['rec1'] = sd3['Confirmed'].shift(periods=28, fill_value=0) #- sd2['Deaths']
sd3['RecEst2'] = sd3['rec1'] - sd3['Deaths']
sd3['dummy'] = 0
sd3['RecEst'] = sd3[["RecEst2", "dummy"]].max(axis=1)
sd3['ActiveEst'] = sd3['Confirmed'] - (sd3['Deaths'] + sd3['RecEst'])



In [ ]:
sd2['rec1'] = sd2['Confirmed'].shift(periods=28, fill_value=0) #- sd2['Deaths']
sd2['RecEst2'] = sd2['rec1'] - sd2['Deaths']
sd2['dummy'] = 0
sd2['RecEst'] = sd2[["RecEst2", "dummy"]].max(axis=1)
sd2['ActiveEst'] = sd2['Confirmed'] - (sd2['Deaths'] + sd2['RecEst'])
sd2

#sd2['dth'] = sd2['DeathDelta'].shift(periods=21, fill_value=0)
#sd2['Rec Delta'] = sd2['rec'] - sd2['DeathDelta']
sd2.tail(50)


# State Level Plot

In [ ]:
# PLOT 3,5 and 7 day moving averages at the State level
import itertools
y= 0.20
colors = itertools.cycle(["deepskyblue","darkblue","green","red"])

sd2["3 day moving"] = sd2["ConfDelta"].rolling(window=3).mean()
sd2["5 day moving"] = sd2["ConfDelta"].rolling(window=5).mean()
sd2["7 day moving"] = sd2["ConfDelta"].rolling(window=7).mean()
#states["3 day moving"] = states["positiveIncrease"].rolling(window=3).mean()
#states["7 day moving"] = states["positiveIncrease"].rolling(window=7).mean()
if sd2["3 day moving"].iloc[(-2)] != 0:
    dt3 = (100*(sd2["3 day moving"].iloc[(-1)]-sd2["3 day moving"].iloc[(-2)])/sd2["3 day moving"].iloc[(-2)]).round(3).astype(str)
else:
    dt3 = "0"
if sd2["5 day moving"].iloc[(-2)] != 0:
    dt5 = (100*(sd2["5 day moving"].iloc[(-1)]-sd2["5 day moving"].iloc[(-2)])/sd2["5 day moving"].iloc[(-2)]).round(3).astype(str)
else:
    dt5 = "0"
if sd2["7 day moving"].iloc[(-2)] != 0:
    dt7 = (100*(sd2["7 day moving"].iloc[(-1)]-sd2["7 day moving"].iloc[(-2)])/sd2["7 day moving"].iloc[(-2)]).round(3).astype(str)
else:
    dt7 = "0"
    
cfr = (100*sd2["Deaths"].iloc[(-1)] / sd2["Confirmed"]).iloc[(-1)].round(1).astype(str)
cases = sd2["Confirmed"].iloc[(-1)].astype(str)
dead = sd2["Deaths"].iloc[(-1)].astype(str)

two_week_change = sd2["7 day moving"].iloc[(-1)] - sd2["7 day moving"].iloc[(-14)] 
sd2b = sd2[sd2['Date']>='2020-03-01']

y= 0.50
x= 0.1
fsize = 14

fig=plt.figure(figsize=(20, 8), dpi= 80, facecolor='w', edgecolor='k')#.legend( prop={'size': 18})

c=next(colors)
plt.plot( 'Date', 'ConfDelta',data=sd2b, marker='', markerfacecolor=c, markersize=12, color=c,linewidth=2)
plt.annotate("Daily Increase: "+ sd2b["ConfDelta"].iloc[(-1)].round(3).astype(str),  color=c,size=fsize, 
             xy=(x, y), 
             xytext=(x, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')

c=next(colors)
plt.plot( 'Date', "3 day moving", data = sd2b , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=3)
y+=0.035
plt.annotate("3 day moving average: "+ sd2b["3 day moving"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt3 +"% )",  color=c,size=fsize, 
             xy=(x, y), 
             xytext=(x, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')


c=next(colors)


plt.plot( 'Date', "5 day moving", data = sd2b , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)

y+=0.035
plt.annotate("5 day moving average: "+ sd2b["5 day moving"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt5 +"% )",  color=c,size=fsize, 
             xy=(x, y), 
             xytext=(x, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')

c=next(colors)
plt.plot( 'Date', "7 day moving", data = sd2b , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=6)
y+=0.035
plt.annotate("7 day moving average: "+ sd2b["7 day moving"].iloc[(-1)].round(3).astype(str) + " (Change: " + dt7 +"% )",  color=c,size=fsize, 
             xy=(x, y), 
             xytext=(x, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')


y+=0.045
plt.annotate("2 Week Change to 7-day moving: "+ two_week_change.round(3).astype(str) ,  color='black' ,size=fsize, 
             xy=(x, y), 
             xytext=(x, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')
y+=0.10
plt.annotate("Total Confirmed Cases: " + cases , color = 'black', size = fsize, xy=(x, y), 
             xytext=(x, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')
y += 0.10
plt.annotate("Total Deaths: " + dead , color = 'black', size = fsize, xy=(x, y), 
             xytext=(x, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')
y += 0.10
plt.annotate("Case Fatality Rate: " + cfr +"%", color = 'black', size = fsize, xy=(x, y), 
             xytext=(x, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')

title = "COVID-19 Daily Positive Infections: State of " + s 

#plt.annotate(s +" 3-day: "+ states[states["state"]==s ]["3 day moving"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
#             xy=(0.15, y), 
#             xytext=(0.15, y),horizontalalignment = 'left',
#             xycoords = 'figure fraction',textcoords='figure fraction')
y+=0.025
plt.title(title, fontsize=24)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Daily New Cases of COVID-19', fontsize=24)
plt.rcParams['xtick.labelsize'] =22
plt.figtext(0.90, 0.01, 'Data gathered by Johns Hopkins University https://github.com/CSSEGISandData ' + now, horizontalalignment='right')
plt.figtext(0.10, 0.01, myname)
plt.tick_params(axis='both', which='minor', labelsize=22)
plt.tick_params(axis='both', which='major', labelsize=22)
plt.grid(which='major',  linestyle='--',linewidth='1')
#plt.grid(which='major', axis='both', linestyle='--',linewidth='1')
#plt.grid(b=True, which='minor', axis='both', linestyle=':', linewidth='2', color='black')

matplotlib.ticker.AutoMinorLocator(n=2)
plt.minorticks_on()
plt.grid( b=True,which='minor', axis='x', color='black', linestyle='-',alpha=0.1)
plt.xticks(fontsize=14, rotation=0)

fn = s + '__daily_case_count_' + nowdate +'.jpg'
plt.savefig(fn)


plt.show()

### State level Active, Recovered, Confirmed, Dead over time

In [ ]:
fig=plt.figure(figsize=(20, 8), dpi= 80, facecolor='w', edgecolor='k')#.legend( prop={'size': 18})
sd2a = sd2[sd2['Date']>='2020-03-01']
y= 0.30
colors = itertools.cycle(["deepskyblue","darkblue","green","red"])
c=next(colors)
plt.plot( 'Date', 'Confirmed',data=sd2a, marker='', markerfacecolor=c, markersize=12, color=c,linewidth=2)
plt.annotate("Confirmed: "+ sd2a["Confirmed"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
             xy=(0.15, y), 
             xytext=(0.15, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')

c=next(colors)
plt.plot( 'Date', "ActiveEst", data = sd2a , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=3)
y+=0.035
plt.annotate("Active: "+ sd2a["ActiveEst"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
             xy=(0.15, y), 
             xytext=(0.15, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')


c=next(colors)


plt.plot( 'Date', "Deaths", data = sd2a , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)

y+=0.035
plt.annotate("Deaths: "+ sd2a["Deaths"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
             xy=(0.15, y), 
             xytext=(0.15, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')
c=next(colors)


plt.plot( 'Date', "RecEst", data = sd2a , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)

y+=0.035
plt.annotate("Recovered(Est.): "+ sd2a["RecEst"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
             xy=(0.15, y), 
             xytext=(0.15, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')
c=next(colors)
plt.plot( 'Date', "ConfDelta", data = sd2a , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=3)
y+=0.035
plt.annotate("New Cases: "+ sd2a["ConfDelta"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
             xy=(0.15, y), 
             xytext=(0.15, y),horizontalalignment = 'left',
             xycoords = 'figure fraction',textcoords='figure fraction')
title = "COVID-19 Daily Situation: State of " + s 
plt.title(title, fontsize=24)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Cases of COVID-19', fontsize=24)
plt.rcParams['xtick.labelsize'] =22
plt.figtext(0.90, 0.01, 'Data gathered by Johns Hopkins University https://github.com/CSSEGISandData ' + now, horizontalalignment='right')
plt.figtext(0.10, 0.01, myname)
plt.tick_params(axis='both', which='minor', labelsize=22)
plt.tick_params(axis='both', which='major', labelsize=22)
plt.grid(which='major',  linestyle='--',linewidth='1')

matplotlib.ticker.AutoMinorLocator(n=2)
plt.minorticks_on()
plt.grid( b=True,which='minor', axis='x', color='black', linestyle='-',alpha=0.1)
plt.xticks(fontsize=14, rotation=0)
plt.show()



In [ ]:
sd2a.tail(15)

In [ ]:
# County level recovery estimate

cd = sd
cd['rec1'] = cd['Confirmed'].shift(periods=28, fill_value=0) #- sd2['Deaths']
cd['RecEst2'] = cd['rec1'] - cd['Deaths']
cd['dummy'] = 0
cd['RecEst'] = cd[["RecEst2", "dummy"]].max(axis=1)
cd['ActiveEst'] = cd['Confirmed'] - (cd['Deaths'] + cd['RecEst'])

In [ ]:
for county in counties:

    cplt1 = cd[cd['Admin2']==county]
    cplt = cplt1[cplt1['Date'] >= '2020-03-01']
    inf_rate = (100000 * cplt["Confirmed"].iloc[(-1)] / cplt["POP_2019"].iloc[(-1)]).round(0).astype(str)

    fig=plt.figure(figsize=(20, 8), dpi= 80, facecolor='w', edgecolor='k')#.legend( prop={'size': 18})
    y= 0.30
    colors = itertools.cycle(["deepskyblue","darkblue","green","red"])
    c=next(colors)
    plt.plot( 'Date', 'Confirmed',data=cplt, marker='', markerfacecolor=c, markersize=12, color=c,linewidth=2)
    plt.annotate("Confirmed: "+ cplt["Confirmed"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')

    c=next(colors)
    plt.plot( 'Date', "ActiveEst", data = cplt , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=3)
    y+=0.035
    plt.annotate("Active: "+ cplt["ActiveEst"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')


    c=next(colors)


    plt.plot( 'Date', "Deaths", data = cplt , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)

    y+=0.035
    plt.annotate("Deaths: "+ cplt["Deaths"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    c=next(colors)


    plt.plot( 'Date', "RecEst", data = cplt , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=4)

    y+=0.035
    plt.annotate("Recovered(Est.): "+ cplt["RecEst"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    c=next(colors)
    plt.plot( 'Date', "ConfDelta", data = cplt , marker='', markerfacecolor=c, markersize=12, color=c,linewidth=3)
    y+=0.035
    plt.annotate("New Cases: "+ cplt["ConfDelta"].iloc[(-1)].round(3).astype(str),  color=c,size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    y+=0.055   
    plt.annotate("Infection Rate per 100K: "+ inf_rate,  color='b',size=20, 
                 xy=(0.15, y), 
                 xytext=(0.15, y),horizontalalignment = 'left',
                 xycoords = 'figure fraction',textcoords='figure fraction')
    
    
    
    
    
    
    title = "COVID-19 Daily Situation: State of " + s + ", " + county + " County"
    plt.title(title, fontsize=24)
    plt.xlabel('Date', fontsize=20)
    plt.ylabel('Cases of COVID-19', fontsize=24)
    plt.rcParams['xtick.labelsize'] =22
    plt.figtext(0.90, 0.01, 'Data gathered by Johns Hopkins University https://github.com/CSSEGISandData ' + now, horizontalalignment='right')
    plt.figtext(0.10, 0.01, myname)
    plt.tick_params(axis='both', which='minor', labelsize=22)
    plt.tick_params(axis='both', which='major', labelsize=22)
    plt.grid(which='major',  linestyle='--',linewidth='1')
    #plt.grid(which='major', axis='both', linestyle='--',linewidth='1')
    #plt.grid(b=True, which='minor', axis='both', linestyle=':', linewidth='2', color='black')

    matplotlib.ticker.AutoMinorLocator(n=2)
    plt.minorticks_on()
    plt.grid( b=True,which='minor', axis='x', color='black', linestyle='-',alpha=0.1)
    plt.xticks(fontsize=14, rotation=0)
    fn = s + '_' + county + '__daily_situation_' + nowdate +'.jpg'
    plt.savefig(fn)
    plt.show()